In [1]:
# Purpose: confirm that the variance files actually contain variance data (and not stddev data)
# If we are targeting for a min SNR of 10, then that means error (stddev) must me 1/10 of the flux, which means variance must be 1/100 of the flux
# that will be our check

# load flux and variance files for 1 observation
# inspect variance header and plot flux/var plot
#   do values make sense?
#   are there any obvious problems?
#   are there any obvious trends?
#   are there any obvious patterns?
#   are there any obvious outliers?
#   are there any obvious problems with the header?
#   What is the variance with in the regions selected or regions of interests?
#    In other words, when looking at the minimum maximum and meaning of the variance, 
#    just consider the variance within the region of interest.

In [2]:
# imports
import sys, os, glob, time
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors

from kcwitools.io import open_kcwi_cube
from kcwitools.utils import build_wave
from kcwitools import image as im


from astropy.wcs import WCS, FITSFixedWarning
import warnings

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore', category=FITSFixedWarning)

bu_path = '/Users/robertseaton/School/github_repos/CGM-learning/code'
if bu_path not in sys.path:
    sys.path.append(bu_path)
from bobutils import utils as bu, fileio as bio, layout_utils as lu, sightlines as bus

global_nb_min = 4676. 
global_nb_max = 4696. 
global_cmap = 'gnuplot'
global_lw = 0.5

In [3]:
def read_and_prep_flux_var_data(flux_file, var_file, nb_index_lo, nb_index_hi):
    """ This method reads the flux and var cubes for a specific observation, and cleans them up before returning """
    hdr_f, fluxcube = open_kcwi_cube(flux_file)
    hdr_v, varcube = open_kcwi_cube(var_file)
    
    wavegrid = build_wave(hdr_f)
    
    slices=(wavegrid >=3500.) & (wavegrid <= 5550.)
    flux=fluxcube[slices,:,:]
    wave=wavegrid[slices]
    var=(varcube[slices,:,:])

    var[np.isnan(flux)]=1.
    flux[np.isnan(flux)]=0.0000

    wave_nb = wave.copy()
    nb_slices=(wavegrid >=nb_index_lo) & (wavegrid <= nb_index_hi)
    wave_nb=wavegrid[nb_slices]

    print("="*50)
    print(f"wave[]:{wave[0]} - {wave[-1]}")
    print(f"wave_nb[]:{wave_nb[0]} - {wave_nb[-1]}")
    print("="*50)

    # Create a narrow-band whitelight image to plot
    whitelight= im.build_whitelight(hdr_f, flux, minwave=wave_nb[0], maxwave=wave_nb[-1])
    return flux, var, wave, whitelight, hdr_f, hdr_v

In [4]:
# show spectra
def show_spectra(ax_spec, combined, color_sightline='blue', color_error='red', color_snr='k', co_begin=4864, co_end=4914, xs=[], ys=[], szs=[]):
    """ show all of the sightline spectra on their own axes """

    spec, var, wave = combined

    ax_spec.plot(wave, spec, '-', color=color_sightline, linewidth=global_lw)
    ax_spec.set_xlabel('wavelength (Angstroms)')
    ax_spec.set_ylabel('Flux', color='k', rotation=90)  # Primary y-axis label, rotated
    ax_spec.tick_params(axis='y', labelcolor='k')
    ax_error = ax_spec.twinx()
    ax_error.plot(wave, var, '-', color=color_error, linewidth=global_lw)
    ax_error.set_ylabel('Error', color='r', rotation=90)  # Secondary y-axis label, rotated
    ax_error.tick_params(axis='y', labelcolor='r')  # Set the tick color to red
    snr2 = bu.sig_figs(bu.signal_to_noise2(wave, spec, co_begin=co_begin, co_end=co_end), 5)
    # snr3 = bu.sig_figs(bu.signal_to_noise3(wave, spec, var, co_begin=co_begin, co_end=co_end), 5)
    # print(f"#0  SNR2: {snr2} SNR3: {snr3}")
    print(f"#0  SNR2: {snr2}")

    ax_spec.text(0.05, 0.9, 
            "Sightline: "+str(0),
            color=color_sightline, 
            fontsize = 12, 
            ha='left', va='top',
            transform=ax_spec.transAxes)
    ax_spec.text(0.05, 0.8, 
            # f"#0  SNR2: {snr2} SNR3: {snr3}",
            f"#0  SNR2: {snr2}",
            color=color_snr, 
            fontsize = 12, 
            ha='left', va='top',
            transform=ax_spec.transAxes)    
    ax_spec.text(0.05, 0.7, 
            f'Coord: ({xs[0]},{ys[0]})',
            color=color_snr, 
            fontsize = 12, 
            ha='left', va='top',
            transform=ax_spec.transAxes)   
    ax_spec.text(0.05, 0.6, 
            f'Aperture: {szs[0]} x {szs[0]}',
            color=color_snr, 
            fontsize = 12, 
            ha='left', va='top',
            transform=ax_spec.transAxes)  
    ax_spec.set_facecolor('darkgrey')

In [5]:
def show_ref_image(ax, image, title="White Light", xh_lim=None, yh_lim=None):
    ax.imshow(image, origin='lower', interpolation='nearest', cmap=global_cmap, vmin=0)
    ax.coords[0].set_ticks_visible(False)  # Hide RA ticks
    ax.coords[1].set_ticks_visible(False)  # Hide Dec ticks
    ax.coords[0].set_ticklabel_visible(False)  # Hide RA tick labels
    ax.coords[1].set_ticklabel_visible(False)  # Hide Dec tick labels
    ax.set_xlabel('')
    ax.set_ylabel('')
    # Optional: Hide the spines
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.set_title(title)
    if xh_lim is not None: ax.set_xlim(xh_lim)
    if yh_lim is not None: ax.set_ylim(yh_lim)
    ax.grid()


In [6]:
def get_single_pointing_data(nb_index_lo, nb_index_hi):
    # get a new flux and variance cube for a single observation
    base_dir="/Users/robertseaton/Library/CloudStorage/GoogleDrive-rwseaton@ncsu.edu/My Drive/Astro Research/KCWI_1429_Data/Fluxed"
    flux_filename = "KB.20180708.23055_icubes_corrected_flux.fits"
    var_filename = "KB.20180708.23055_icubes_corrected_var.fits"

    flux_file = os.path.join(base_dir, flux_filename)
    var_file = os.path.join(base_dir, var_filename)

    oflux, ovar, owave, owl, ohdr_f, ohdr_v = read_and_prep_flux_var_data(flux_file, var_file, nb_index_lo, nb_index_hi)
    return oflux, ovar, owave, owl, ohdr_f, ohdr_v


In [8]:

def main():
    # define sightline coordinates + aperture sizes
    xs, ys, szs = bus.load_sightlines()
    wl_image, wcs_ref = bio.load_original_cube(global_nb_min, global_nb_max)
    sl_radecs = bus.radecs_from_sightline_boxes(wcs_ref, xs, ys, szs)

    
    # draw whitelight image
    fig = plt.figure(figsize=(10,7), dpi=150)
    gs = gridspec.GridSpec(4, 2, figure=fig)
    ax_ref_image = fig.add_subplot(gs[0:2, 0:1], projection=wcs_ref)
    
    show_ref_image(ax_ref_image, wl_image, title="White Light", xh_lim=[0,100], yh_lim=[0,100])
    # oflux, ovar, owave, owl, ohdr_f, ohdr_v = get_single_pointing_data(global_nb_min, global_nb_max)
    # show_ref_image(ax_ref_image, wl_image, title="White Light Image")
    # get flux and variance cube for a single observation
    # draw the flux and variance image
    # draw the spectra for the sightlines
    # draw the spectra for the sightlines with the variance
    plt.show()
    
    
main()

=== sightline   pt_xs: 29 pt_ys: 36 szs: 3
=== sightline box corners   xs: [26.5, 29.5] ys: [33.5, 36.5]
radec: (array([217.47918507, 217.47893673]), array([12.04366569, 12.04390857]))
